In [ ]:
import lyricsgenius
import pandas as pd

# Tu token de autenticación (Bearer Token)
token = "tyVAyvo90CltRuI4bfPbK4VS_dipGtrHFtIPWtaY_NRDkuvGUpsG158goOTn"

# Inicializa la instancia de Genius
genius = lyricsgenius.Genius(token)

In [ ]:
import pandas as pd
import lyricsgenius
from joblib import Parallel, delayed
from tqdm import tqdm
import time

def fetch_lyrics(track_name, artist_name, genius, no_lyrics):
    try:
        song = genius.search_song(track_name, artist_name)
        if song:
            return song.lyrics
        else:
            no_lyrics.append((track_name, artist_name))
            return None
    except:
        no_lyrics.append((track_name, artist_name))
        return None

def process_batch(batch_df, genius, no_lyrics, pause_time):
    lyrics = []
    for index, row in batch_df.iterrows():
        lyrics.append(fetch_lyrics(row['track_name'], row['artist_name'], genius, no_lyrics))
        if pause_time > 0:
            time.sleep(pause_time)  # Pausa entre solicitudes para no sobrecargar la API
    return pd.Series(lyrics, index=batch_df.index)

def get_lyrics(df, batch_size=200, pause_time=0.2, n_jobs=6, token="NTWXUPKn3DXfY9SIpoX30G0HVcjH3GSskIk0KURIsxXQaXjCr0ljIq5IyLA5y7p0"):
    # Inicializa la instancia de Genius
    genius = lyricsgenius.Genius(token)
    no_lyrics = []

    # Dividir el DataFrame en lotes
    batches = [df.iloc[i:i + batch_size] for i in range(0, len(df), batch_size)]
    total_batches = len(batches)
    half_batches = total_batches // 2

    # Procesar cada lote en paralelo
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_batch)(batch, genius, no_lyrics, pause_time) for batch in tqdm(batches, desc="Procesando lotes")
    )
    
    # Imprimir progreso al 50%
    if half_batches > 0:
        print("50% del procesamiento completado.")

    # Concatenar los resultados de todos los lotes en una sola columna
    lyrics_series = pd.concat(results).reset_index(drop=True)
    df = df.reset_index(drop=True)
    df['lyrics'] = lyrics_series
    
    return df, no_lyrics

In [ ]:
df_tracks = pd.read_csv("data/tracks.csv")

# Llamar a la función get_lyrics
df_tracks_lyrics, no_lyrics = get_lyrics(df_tracks, batch_size=100, pause_time=0.5, n_jobs=8, token="NTWXUPKn3DXfY9SIpoX30G0HVcjH3GSskIk0KURIsxXQaXjCr0ljIq5IyLA5y7p0")

In [ ]:
df_tracks_lyrics.to_json("data/more_playlists.json")